<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/4/47/Acronimo_y_nombre_uc3m.png" width=50%/>

<h1><font color='#12007a'>Procesamiento de Lenguaje Natural con Aprendizaje Profundo</font></h1>
<p>Autora: Isabel Segura Bedmar</p>

<img align='right' src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" width=15%/>
</center>    

# 7.2. Reconocimiento de entidades con BERT.

En este ejercicio, vamos a desarrollar un enfoque basado en el transformer BERT para la tarea de reconocimiento de entidades (named entity recognition, NER). El objetivo de esta tarea es encontrar entidades nombradas (como personas, localizaciones u organizaciones) en un texto. El conjunto de tipos de entidades depende del dominio. Así, por ejemplo, para un dominio biomédico, los tipos de entidad podrían ser: fármaco, gen, enfermedad, etc.

NER se puede ver como una tarea de clasificación de tokens, donde el objetivo será asignar una etiqueta a cada token. En concreto, las etiquetas del estándar IOB, donde
- B-type representa un token que es el primer token de una entidad de tipo type.  
- I-type representa un token que pertenece a una entidad, pero no es el primero de sus tokens.
- O se utiliza para representar tokens que no pertenecen a ninguna entidad.



En este ejercicio, vamos a utilizar como dataset uno de los más populares para la tarea de NER: CoNLL-2003 (https://huggingface.co/datasets/conll2003). Este dataset es una colección de noticias de Reuters anotadas con entidades como Persona, Ubicación, Organización y Varios.



## Instalar las librerías

In [1]:
!pip install -q datasets transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.8 MB/s eta 0:00:00


## Cargar el dataset


In [2]:
from datasets import load_dataset
dataset_dict = load_dataset("conll2003")
dataset_dict

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

Podemos ver que el dataset es distribuido con tres splits: train, validation y test.

Cada instancia contiene las siguientes características:

- id: el identificador del texto
- tokens: la lista de tokens del texto.
- pos_tags: la lista de etiquetas PoS (categoría gramátical) de los tokens del texto.
- chunk_tags: la lista de etiquetas en formato IOB para representar sintagmas nominales.
- ner_tags: la lista de etiquetas NER, en formatio IOB, para los tokens en el texto.

REspecto al conjunto de entidades (ner_tags), veremos que en las instancias ya están coficiadas con número enteros, pero el conjunto inicial de etiquetas puede obtenerse consultando el campo features para **ner_tags**:

In [11]:
ner_tags = dataset_dict["train"].features["ner_tags"]
LABELS = ner_tags.feature.names
LABELS

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

Así cada label está codificada con un entero:


In [16]:
idx2label={}
label2idx={}
for index, label in enumerate(LABELS):
    print(label)
    label2idx.update([(label, index)])
    idx2label.update([(index, label)])

print(label2idx)

O
B-PER
I-PER
B-ORG
I-ORG
B-LOC
I-LOC
B-MISC
I-MISC
{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}


Vamos a ver una instancia y comprobamos si sus labels (enteros) son correctos:

In [3]:
dataset_dict["train"][0]["tokens"]

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']

Vamos a ver también sus etiquetas

In [14]:
dataset_dict["train"][0]["ner_tags"]

[3, 0, 7, 0, 0, 0, 7, 0, 0]

Ejecuta esta celda varias veces y podrás ver distintas oraciones con su etiquetado para la tarea NER:

In [27]:
import random
index = random.randint(0,dataset_dict["train"].num_rows)
tokens = dataset_dict["train"][index]["tokens"]
tags = dataset_dict["train"][index]["ner_tags"]

text = ""
annotated_text = ""

for token, tag in zip(tokens, tags):
    text += token + " "
    text_label = idx2label[tag]
    annotated_text += str(text_label) + "   "

print(text)
print(annotated_text)

Santos suffered more from their loss as Narciso 's replacement Jean gave away a penalty from which Sao Paulo scored the decisive goal in a 2-1 win . 
B-ORG   O   O   O   O   O   O   B-PER   O   O   B-PER   O   O   O   O   O   O   B-ORG   I-ORG   O   O   O   O   O   O   O   O   O   


## Tokenización
Como de costumbre, necesitamos preprocesar los textos y transformarlos en el formato que necesita BERT: (input_ids, input_type_ids y attention_mask).


Para ello necesitamos cargar el tokenizador del modelo BERT. En este caso, usaremos la versión cased del modelo BERT base. La versión cased es mejor para tareas NER, mientras que uncased es mejor para tareas de clasificación de textos.

In [28]:
from transformers import AutoTokenizer

model_name = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [29]:
tokenizer.is_fast

True

Aunque los textos en del dataset ya están tokenizadors, necesitamos codificarlos para preparar la entrada para BERT.
Afortunadamente, el tokenizador de BERT, también nos permite tokenizar tokens (en lugar de textos), gracias al parámetro **is_split_into_words = True**.

Vamos a ver un ejemplo, antes de aplicar el tokenizador a toda la colección. En la siguiente celda, codificamos los tokens del primer texto. El método tokens() nos permite obtener la lista de tokens.

Podemos ver que se han agregado dos tokens especiales, [CLS] y [SEP]. Además, las palabras que no están en el vocabulario se han dividido en subpalabras. Por ejemplo, 'lamb' se dividió en 'la' y '##mb'.

In [30]:
inputs = tokenizer(dataset_dict["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'EU',
 'rejects',
 'German',
 'call',
 'to',
 'boycott',
 'British',
 'la',
 '##mb',
 '.',
 '[SEP]']

El método word_ids() nos permite conocer el índice de cada token en el texto original. Los tokens especiales están indexados con None. El token 'EU' tiene el id 0, mientras que los tokens 'la' y '##mb" corresponden a la palabra en la posición 7.

In [ ]:
inputs.word_ids()

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]

Así podemos ver que mientras el tokenizador devuelve 12 tokens, en realidad, el texto original únicamente tiene 10 tokens:

In [34]:
print('Tamaño de word_ids:', len(inputs.word_ids()))
#quitamos el 1 de None
print('Número de palabras en el texto original:', len(set(inputs.word_ids())) - 1)

Tamaño de word_ids: 12
Número de palabras en el texto original: 9


Ahora vamos a ver sus etiquetas NER:

In [33]:
tags = dataset_dict["train"][0]["ner_tags"]
print(tags)
print("Number of tokens", len(tags))


[3, 0, 7, 0, 0, 0, 7, 0, 0]
Number of tokens 9


Aquí nos encontramos con un problema, ya que mientras el tokenizador devuelve una lista de 12 tokens, el dataset únicamente proporciona 9 etiquetas NER.

El culpable es el tokenizador de BERT, porque:
- ha añadido los tokens CLS y SEP, para los que el dataset no está dando ninguna etiqueta.
- además, el tokenizador ha dividido lamb en dos subtokens, pero el dataset únicamente proporciona una etiqueta.


Vamos a definir una función que asigne correctamente a cada token/subtoken su etiqueta NER correspondiente. En el ejemplo anterior, tanto 'la' como '#mb" deben anotarse con O.

In [36]:
def align_labels_with_tokens(word_ids, tags):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id is None:
            new_labels.append(-100)     # tokens especials se codifican con -100
        elif word_id != current_word:   # pasamos a un nuevo token
            current_word = word_id
            new_labels.append(tags[word_id])
        else:                           # estamos en el mismo token
            label = tags[current_word]
            # ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']
            #  0        1       2       3       4           5       6       7           8
            if label % 2 != 0:       # si la label es impar, quiere. decir que el anterior era el primero,
                                     # debemos pasar a I- (es la siguiente)
                new_labels.append(label+1)
            else:
                new_labels.append(label)


    return new_labels

Vamos a comprobarlo para las cinco primeras oraciones del split de training:

In [38]:
for i in range(5):
    print("\nSentence: ", str(i))
    tags = dataset_dict["train"][i]["ner_tags"]
    inputs = tokenizer(dataset_dict["train"][i]["tokens"], is_split_into_words=True)
    word_ids = inputs.word_ids()

    aligned_labels = align_labels_with_tokens(word_ids, tags)
    print(inputs.tokens())
    print(aligned_labels)
    for t in aligned_labels:
        if t != -100:
            print(idx2label[t], end = ' ')
        else:
            print(str(-100), end = ' ')

    print()





Sentence:  0
['[CLS]', 'EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'la', '##mb', '.', '[SEP]']
[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]
-100 B-ORG O B-MISC O O O B-MISC O O O -100 

Sentence:  1
['[CLS]', 'Peter', 'Blackburn', '[SEP]']
[-100, 1, 2, -100]
-100 B-PER I-PER -100 

Sentence:  2
['[CLS]', 'BR', '##US', '##SE', '##LS', '1996', '-', '08', '-', '22', '[SEP]']
[-100, 5, 6, 6, 6, 0, 0, 0, 0, 0, -100]
-100 B-LOC I-LOC I-LOC I-LOC O O O O O -100 

Sentence:  3
['[CLS]', 'The', 'European', 'Commission', 'said', 'on', 'Thursday', 'it', 'disagreed', 'with', 'German', 'advice', 'to', 'consumers', 'to', 's', '##hun', 'British', 'la', '##mb', 'until', 'scientists', 'determine', 'whether', 'mad', 'cow', 'disease', 'can', 'be', 'transmitted', 'to', 'sheep', '.', '[SEP]']
[-100, 0, 3, 4, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]
-100 O B-ORG I-ORG O O O O O O B-MISC O O O O O O B-MISC O O O O O O O O O O O O O O O -10

La siguiente función recibe como entrada un conjunto de datos, aplica el tokenizador y alinea las etiquetas del dataset con esta tokenización.
Esta función añade un nuevo campo al conjunto de datos que contiene las etiquetas alineadas (lo vamos a llamar labels).

In [41]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        #get the ids for the instance i
        word_ids = tokenized_inputs.word_ids(i)
        #align the words with their corresponding labels
        new_labels.append(align_labels_with_tokens(word_ids, labels))

    # we add a new feature to the dataset with the aligned labels for each instance
    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

train_encodings = tokenize_and_align_labels(dataset_dict['train'])


Podemos aplicar la función anterior a todo el dataset. También podemos eliminar todas las columnas (excepto la de labels)

In [42]:
tokenized_datasets = dataset_dict.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset_dict["train"].column_names,
)
tokenized_datasets

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})

Si mostramos las cnco primeras oraciones del training, podemos ver que tienen tamaños distintos:

In [44]:
for i in range(5):
    print(tokenized_datasets["train"][i]["input_ids"])
    print(tokenized_datasets["train"][i]["labels"])

[101, 7270, 22961, 1528, 1840, 1106, 21423, 1418, 2495, 12913, 119, 102]
[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]
[101, 1943, 14428, 102]
[-100, 1, 2, -100]
[101, 26660, 13329, 12649, 15928, 1820, 118, 4775, 118, 1659, 102]
[-100, 5, 6, 6, 6, 0, 0, 0, 0, 0, -100]
[101, 1109, 1735, 2827, 1163, 1113, 9170, 1122, 19786, 1114, 1528, 5566, 1106, 11060, 1106, 188, 17315, 1418, 2495, 12913, 1235, 6479, 4959, 2480, 6340, 13991, 3653, 1169, 1129, 12086, 1106, 8892, 119, 102]
[-100, 0, 3, 4, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]
[101, 1860, 112, 188, 4702, 1106, 1103, 1735, 1913, 112, 188, 27431, 3914, 14651, 163, 7635, 4119, 1163, 1113, 9031, 11060, 1431, 4417, 8892, 3263, 2980, 1121, 2182, 1168, 1190, 2855, 1235, 1103, 3812, 5566, 1108, 27830, 119, 102]
[-100, 5, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, -100]


### Data collator

En lugar de usar padding en la tokenización, esta vez vamos a almacenar los datos en un colector de datos (**datacollector**). En concreto, vamos a usar la clase **DataCollatorForTokenClassification** que directamente añade padding a todos los campos de la entrada:  input_ids, token_type_ids, attention_mask y labels.



In [45]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)



You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0,    0, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100],
        [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100],
        [-100,    5,    6,    6,    6,    0,    0,    0,    0,    0, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100],
        [-100,    0,    3,    4,    0,    0,    0,    0,    0,    0,    7,    0,
            0,    0,    0,    0,    0,    7,    0,    0,    0,    0,    0,    0,
            0,    0,    0

Aplicamos el colector primero a los cinco primeras instancias del training:

In [46]:

batch = data_collator([tokenized_datasets["train"][i] for i in range(5)])
batch["labels"]


tensor([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0,    0, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100],
        [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100],
        [-100,    5,    6,    6,    6,    0,    0,    0,    0,    0, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100],
        [-100,    0,    3,    4,    0,    0,    0,    0,    0,    0,    7,    0,
            0,    0,    0,    0,    0,    7,    0,    0,    0,    0,    0,    0,
            0,    0,    0

Podemos ver que ahora todos las secuencias tienen la misma longitud (en las más cortas se ha añadido -100 hasta alcanzar la longitud máxima en el lote).

## Modelo

### Metricas e hiper-parámetros
Necesitamos definir las métricas que el modelo debe calcular en cada época sobre el conjunto de validación.

Vamos a implementar una función **compute_metrics()** que reciba las predicciones y sus correspondientes etiquetas gold-standard, y cálcula las métricas más apropiadas para la tarea (en concreto, precision, recall y f1).

La función devuelve un diccionario con los nombres de las métricas y sus puntuaciones.

Vamos a  usar la librería **seqeval** que permite calcular fácilmente esas métricas para cada uno de las etiquetas:


In [48]:
!pip install -q seqeval Evaluate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.7 MB/s eta 0:00:00


In [54]:
import evaluate

metric = evaluate.load("seqeval")

import numpy as np

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Ignoramos los tokens especiales (-100)
    true_labels = [[idx2label[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [idx2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }


Recuerda que también tenemos que definir el conjunto de hiperparámetros:


In [55]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir='./outputs/',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=1,  # 1
    weight_decay=0.01,
)

### Modelo
Como nuestra tarea es un problema de clasificación de tokens, vamos a usar la clase **AutoModelForTokenClassification**.


In [56]:
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    # instead of using num_labels, for NER is better to provide the correspondence between labels and their indexes
    id2label=idx2label,
    label2id=label2idx,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Vamos a comprobar que en efecto el número de labels es 9:

In [57]:
model.config.num_labels


9

### Training
Ahora vamos a entrenar el modelo usando la clase Trainer, que como ya hemos visto en ejercicios anteriores, nos va a facilitar enormemente el desarrollo del ciclo de entrenamiento.

Recuerda comprobar que tu entorno de ejecución usa GPU o TPU:

In [58]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.078100,0.065363,0.909299,0.934702,0.921826,0.981868


TrainOutput(global_step=1756, training_loss=0.13596910333307566, metrics={'train_runtime': 206.5341, 'train_samples_per_second': 67.984, 'train_steps_per_second': 8.502, 'total_flos': 307479492426330.0, 'train_loss': 0.13596910333307566, 'epoch': 1.0})

### Evaluación
Podemos ver los resultados finales sobre el conjunto de validación:

In [ ]:
trainer.evaluate()

{'eval_loss': 0.06859466433525085,
 'eval_precision': 0.9013854930725347,
 'eval_recall': 0.9306630764052508,
 'eval_f1': 0.915790345284425,
 'eval_accuracy': 0.9815888620709955,
 'eval_runtime': 10.7052,
 'eval_samples_per_second': 303.591,
 'eval_steps_per_second': 38.019,
 'epoch': 1.0}

## Evaluación sobre el conjunto test

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [[idx2label[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

true_labels = [[idx2label[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

results = metric.compute(predictions=true_predictions, references=true_labels)

results


{'LOC': {'precision': 0.9016393442622951,
  'recall': 0.9232613908872902,
  'f1': 0.9123222748815166,
  'number': 1668},
 'MISC': {'precision': 0.6719706242350061,
  'recall': 0.782051282051282,
  'f1': 0.7228439763001974,
  'number': 702},
 'ORG': {'precision': 0.8384481760277939,
  'recall': 0.8717639975918121,
  'f1': 0.8547815820543093,
  'number': 1661},
 'PER': {'precision': 0.9446135118685332,
  'recall': 0.9598021026592455,
  'f1': 0.9521472392638036,
  'number': 1617},
 'overall_precision': 0.8632739609838846,
 'overall_recall': 0.9010269121813032,
 'overall_f1': 0.8817465130382051,
 'overall_accuracy': 0.9696033784529081}

También podemos mostrar los resultados a nivel de token:

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(np.concatenate(true_labels), np.concatenate(true_predictions)))

              precision    recall  f1-score   support

       B-LOC       0.93      0.93      0.93      1668
      B-MISC       0.79      0.82      0.80       702
       B-ORG       0.91      0.90      0.91      1661
       B-PER       0.96      0.96      0.96      1617
       I-LOC       0.91      0.90      0.90      1748
      I-MISC       0.58      0.63      0.60       886
       I-ORG       0.89      0.91      0.90      3172
       I-PER       0.97      0.97      0.97      4082
           O       0.99      0.99      0.99     47925

    accuracy                           0.97     63461
   macro avg       0.88      0.89      0.89     63461
weighted avg       0.97      0.97      0.97     63461



Los resultados a nivel entidad son:

In [ ]:
from seqeval.metrics import classification_report as classification_report_seqeval
print(classification_report_seqeval(true_labels, true_predictions))


              precision    recall  f1-score   support

         LOC       0.90      0.92      0.91      1668
        MISC       0.67      0.78      0.72       702
         ORG       0.84      0.87      0.85      1661
         PER       0.94      0.96      0.95      1617

   micro avg       0.86      0.90      0.88      5648
   macro avg       0.84      0.88      0.86      5648
weighted avg       0.87      0.90      0.88      5648

